In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import glob
import csv
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import mpl_toolkits.mplot3d as mp3d

path = r'C:\Users\tiany\Documents\EE6895 big data\bill\110'
allFiles = glob.glob(path + "/*.csv")
#allFiles = glob.glob("bill/*.csv")
#create lists
#vote same or not between party and individuals

#try legislator 'Neil Abercrombie''Democrate'/ 'Stephanie Herseth Sandlin

dfn = pd.read_csv(allFiles[1],  encoding='latin-1',low_memory=False)
name = dfn.columns.values.tolist()
v = name[3]
dfn = dfn.rename(columns={v:'Vote'})
dfn = dfn.drop_duplicates(subset=['Legislator'])
dfn = dfn['Legislator'].tolist()
corrupt_chance = []
legislator = []
for k in range(len(dfn)):
    dfn[k] = dfn[k].replace('"', '')
    
for u in range(40):
    Vote_condition = []
    #contribution differences
    difference = []
    #bill number
    billname = []
    corrupt_point = []
    for i in range(len(allFiles)):
        df = pd.read_csv(allFiles[i], encoding='latin-1',low_memory=False)
        df['Contribution Amount'] = df['Contribution Amount'].str.replace('$','')
        df['Contribution Amount'] = df['Contribution Amount'].astype(float)
        name = df.columns.values.tolist()
        v = name[3]
        n = name[-4]
        df = df.rename(columns={n: 'Interest Group Position',v:'Vote'})
        clean = df.drop(['Represents','Contribution Type','Presidential','Contribution Date','Contributor','Contributor Occupation','Contributor Interest Group Code','Contributor City','Contributor State','Contributor Zip','Contributor Employer','Contributor Interest Group'],axis=1)
        g = clean.groupby(['Party','Interest Group Position']).sum()
        newclean =  df.drop(['Represents','Contribution Type','Presidential','Contribution Amount','Contribution Date','Contributor','Contributor Occupation','Contributor Interest Group Code','Contributor City','Contributor State','Contributor Zip','Contributor Employer','Contributor Interest Group'],axis=1)
        dd = newclean.drop_duplicates(subset=['Legislator', 'Party', 'Vote'])
        a = (dd['Party'][dd['Vote']=='No']=='Democrat').sum()
        b = (dd['Party'][dd['Vote']=='Not Voting']=='Democrat').sum()
        c = (dd['Party'][dd['Vote']=='Yes']=='Democrat').sum()
        d = (dd['Party'][dd['Vote']=='No']=='Independent').sum()
        e = (dd['Party'][dd['Vote']=='Not Voting']=='Independent').sum()
        f = (dd['Party'][dd['Vote']=='Yes']=='Independent').sum()
        j = (dd['Party'][dd['Vote']=='No']=='Republican').sum()
        k = (dd['Party'][dd['Vote']=='Not Voting']=='Republican').sum()
        l = (dd['Party'][dd['Vote']=='Yes']=='Republican').sum()

        #dataframe
        pd.set_option('display.precision',15)
        pd.DataFrame({'x':[.000000000000001]})

        arrays = [['Democrat', 'Democrat','Democrat', 'Republican','Republican', 'Republican'],
                              ['Oppose','', 'Support', 'Oppose','', 'Support']]
        tuples = list(zip(*arrays))
        index = pd.MultiIndex.from_tuples(tuples, names=['Party', 'Interest Group Position'])
        column = ['Contribution Amount', 'Vote', 'Count']
        s = pd.DataFrame(index=index, columns=column)
        votes = ['No','Not Voting','Yes','No','Not Voting','Yes']
        count = [a,b,c,j,k,l]
        s['Vote'] = votes
        s['Count'] = count
        newframe = s.fillna(g)
        effect = newframe.fillna(0)
        gg = clean.groupby(['Legislator','Party','Vote','Interest Group Position']).sum()
        leglist = newclean.drop_duplicates(subset=['Legislator'])
        leglist = leglist['Legislator'].tolist()
        num = len(leglist)
        reset = gg.reset_index()
        try:
            name = reset[reset['Legislator']==dfn[u]]
            if name['Party'].any() == 'Democrat':
                con_oppo = name['Contribution Amount'][name['Interest Group Position']=='Oppose'].tolist()
                if con_oppo ==[]:
                    con_oppo = 0.0
                else:
                    con_oppo = con_oppo[0]
                con_sup = name['Contribution Amount'][name['Interest Group Position']=='Support'].tolist()
                if con_sup ==[]:
                    con_sup = 0.0
                else:
                    con_sup = con_sup[0]
                #party vote no
                if a > c:
                    if name.iloc[0]['Vote'] =='No':
                        Vote_condition.append(1)
                        dif = con_sup - con_oppo
                        #dif = dif/1000
                        difference.append(dif)
                    if name.iloc[0]['Vote'] =='Yes':
                        Vote_condition.append(-1)
                        dif = con_sup - con_oppo
                        #dif = dif/1000
                        difference.append(dif)
                        if dif > 0:
                            corrupt_point.append(dif)
                #party vote yes
                if a < c:
                    if name.iloc[0]['Vote'] =='Yes':
                        Vote_condition.append(1)
                        dif = con_oppo - con_sup 
                        #dif = dif/1000
                        difference.append(dif)
                    if name.iloc[0]['Vote'] =='No':
                        Vote_condition.append(-1)
                        dif = con_oppo - con_sup 
                        #dif = dif/1000
                        difference.append(dif)
                        if dif > 0:
                            corrupt_point.append(dif)
            if name['Party'].any() == 'Republican':
                con_oppo = name['Contribution Amount'][name['Interest Group Position']=='Oppose'].tolist()
                if con_oppo ==[]:
                    con_oppo = 0.0
                else:
                    con_oppo = con_oppo[0]
                con_sup = name['Contribution Amount'][name['Interest Group Position']=='Support'].tolist()
                if con_sup ==[]:
                    con_sup = 0.0
                else:
                    con_sup = con_sup[0]
                #party vote no
                if j > l:
                    if name.iloc[0]['Vote'] =='No':
                        Vote_condition.append(1)
                        dif = con_sup - con_oppo
                        #dif = dif/1000
                        difference.append(dif)
                    if name.iloc[0]['Vote'] =='Yes':
                        Vote_condition.append(-1)
                        dif = con_sup - con_oppo
                        #dif = dif/1000
                        difference.append(dif)
                        if dif > 0:
                            corrupt_point.append(dif)
                #party vote yes
                if j < l:
                    if name.iloc[0]['Vote'] =='Yes':
                        Vote_condition.append(1)
                        dif = con_oppo - con_sup 
                        #dif = dif/1000
                        difference.append(dif)
                    if name.iloc[0]['Vote'] =='No':
                        Vote_condition.append(-1)
                        dif = con_oppo - con_sup 
                        #dif = dif/1000
                        difference.append(dif)
                        if dif > 0:
                            corrupt_point.append(dif)
        except:
            pass
    for j in range(len(difference)):
        billname.append(j)
    try:    
        corrupt_rate = len(corrupt_point)/len(billname)
        #print(dfn[u], corrupt_rate)
        corrupt_chance.append(corrupt_rate)
        legislator.append(dfn[u])


       
    except:
        pass
cr = {'Legislator': legislator, 'corrupt rate':corrupt_chance}
crdf = pd.DataFrame(cr)

C:\Users\tiany\Anaconda3\lib\site-packages\pandas\core\ops.py:798: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(x, name)(y)


In [12]:
#crdf['corrupt_Rank'] = crdf.groupby('Legislator')['corrupt rate'].rank()
crdf['corrupt_Rank'] = crdf['corrupt rate'].rank()

In [17]:
crdf

In [31]:
pathf = r'C:\Users\tiany\Documents\EE6895 big data\bill\ '
crdf.to_csv(pathf+'rank.csv') 